<a href="https://colab.research.google.com/github/stelios191/Agenda-Based-Search-and-Genetic-Algorithm/blob/main/Agenda_based_search_and_genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Agenda based search
*AI route finder using an agenda-based search mechanism for London Tube map.


In [ ]:
import pandas as pd
from collections import deque
from queue import PriorityQueue
df = pd.read_csv('tubedata.csv', header=None)
df.head()
from collections import defaultdict
station_dict = defaultdict(list)
zone_dict = defaultdict(set)
line_dict = defaultdict(set)
# get data row by row
for index, row in df.iterrows():
  start_station = row[0]
  end_station = row[1]
  line = row[2]  # Include the tube line
  act_cost = int(row[3])
  zone1 = row[4]
  zone2 = row[5]
  # Update line dictionary for both start and end stations
  line_dict[start_station].add(line)
  line_dict[end_station].add(line)
  # station dictionary of child station tuples
  # (child_name, cost from parent to the child)
  # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
  station_list = station_dict[start_station]
  station_list.append((end_station, act_cost))#tubeline
  # the following two lines add the other direction of the tube "step"
  station_list = station_dict[end_station]
  station_list.append((start_station, act_cost))#tubeline
  # we add the main zone
  zone_dict[start_station].add(zone1)
  # we add the secondary zone
  if zone2 != "0":
    zone_dict[start_station].add(zone2)
    # if the secondary zone is not 0 it’s the main zone for the ending station
    zone_dict[end_station].add(zone2)
  else:
    # otherwise the main zone for the ending station
    # is the same as for the starting station
    zone_dict[end_station].add(zone1)

In [ ]:
class TubeState:
    """
    Class to represent a state for the for search algorithms

    Attributes:
    - current_station (str): The current station
    - path (list): The list of stations visited so far, including the current station
    - total_cost (int): The cumulative average time taken to reach this state
    - visited_stations (set): A set of stations that have been visited so far
    - last_line (str or None): The last line taken to reach the current station
    """
    def __init__(self, current_station, path=None, total_cost=0, visited_stations=None,last_line=None):
        self.current_station = current_station
        self.path = path if path is not None else [current_station]
        self.total_cost = total_cost
        self.visited_stations = visited_stations if visited_stations is not None else set(self.path)
        self.last_line=last_line

    def __str__(self):
        return f"Station: {self.current_station}, Path: {self.path}, Cost: {self.total_cost}"

    def expand(self, station_dict, line_dict):
        """
        expand the current state to generate new states for each unvisited neighboring
        Args:
          -station_dict: The dictionary containing station connectivity and costs
          -line_dict : The dictionary containing the lines each station belongs to
        returns:
        - list of new TubeState instances representing the neighboring states
        """
        new_states = []
        for neighbor, cost in station_dict[self.current_station]:
            if neighbor not in self.visited_stations:
                new_path = self.path + [neighbor]
                new_visited_stations = self.visited_stations.union({neighbor})
                new_last_line = self.last_line

                # assign a line only if it is not the first expansion
                if len(self.path) > 1:
                    new_last_line = self.determine_line_used(neighbor, line_dict)

                new_states.append(TubeState(neighbor, new_path, self.total_cost + cost, new_visited_stations, new_last_line))
        return new_states
      # comparison based on total_cost
    def __lt__(self, other):

        return self.total_cost < other.total_cost
    # function to check for line changes
    def determine_line_used(self, next_station, line_dict):
        current_lines = line_dict[self.current_station]
        next_lines = line_dict[next_station]
        common_lines = current_lines.intersection(next_lines)
        if common_lines:
            return common_lines.pop()  # Return one of the common lines
        else:
            return next_lines.pop() if next_lines else None


In [ ]:
def dfs(start_station, end_station, station_dict):
    """
    dfs

    Args:
    - start_station (str): The starting station.
    - end_station (str): The destination station.
    - station_dict (dict): The dictionary containing station connectivity

    Returns:
    - Tuple: (path, total_cost, number_of_expanded_nodes)
    """
    initial_state = TubeState(start_station)
    frontier = [initial_state]
    visited = set()  # Set to keep track of visited nodes to avoid loop
    number_of_expanded_nodes = 0
    while frontier:
        current_state = frontier.pop()

        # If the current station is the goal, return the result
        if current_state.current_station == end_station:
            return (current_state.path, current_state.total_cost, number_of_expanded_nodes)

        # Explore neighboring stations that have not been visited and add them to the queue
        if current_state.current_station not in visited:
            visited.add(current_state.current_station)
            number_of_expanded_nodes += 1
            for new_state in reversed(current_state.expand(station_dict,line_dict)):
                if new_state.current_station not in visited:
                    frontier.append(new_state)

    return (None, None, number_of_expanded_nodes)
# Route from Euston to Victoria
dfs_path, dfs_cost, dfs_expanded_nodes = dfs("Euston", "Victoria", station_dict)
print("Euston to Victoria:")
print("Path:", dfs_path)
print("Cost:", dfs_cost)
print("Expanded Nodes:", dfs_expanded_nodes)
print()

# Route from Canada Water to Stratford
dfs_path, dfs_cost, dfs_expanded_nodes = dfs("Canada Water", "Stratford", station_dict)
print("Canada Water to Stratford:")
print("Path:", dfs_path)
print("Cost:", dfs_cost)
print("Expanded Nodes:", dfs_expanded_nodes)
print()

# Route from Baker Street to Wembley Park
dfs_path, dfs_cost, dfs_expanded_nodes = dfs("Baker Street", "Wembley Park", station_dict)
print("Baker Street to Wembley Park:")
print("Path:", dfs_path)
print("Cost:", dfs_cost)
print("Expanded Nodes:", dfs_expanded_nodes)
print()

# Route from New Cross Gate to Stepney Green
dfs_path, dfs_cost, dfs_expanded_nodes = dfs("New Cross Gate", "Stepney Green", station_dict)
print("New Cross Gate to Stepney Green:")
print("Path:", dfs_path)
print("Cost:", dfs_cost)
print("Expanded Nodes:", dfs_expanded_nodes)
print()

# Route from Ealing Broadway to South Kensington
dfs_path, dfs_cost, dfs_expanded_nodes = dfs("Ealing Broadway", "South Kensington", station_dict)
print("Ealing Broadway to South Kensington:")
print("Path:", dfs_path)
print("Cost:", dfs_cost)
print("Expanded Nodes:", dfs_expanded_nodes)


Euston to Victoria:
Path: ['Euston', 'Warren Street', 'Goodge Street', 'Tottenham Court Road', 'Oxford Circus', "Regent's Park", 'Baker Street', 'Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush", 'White City', 'East Acton', 'North Acton', 'West Acton', 'Ealing Broadway', 'Ealing Common', 'Acton Town', 'Chiswick Park', 'Turnham Green', 'Stamford Brook', 'Ravenscourt Park', 'Hammersmith', 'Barons Court', 'West Kensington', "Earls' Court", 'High Street Kensington', 'Gloucester Road', 'South Kensington', 'Sloane Square', 'Victoria']
Cost: 69
Expanded Nodes: 87

Canada Water to Stratford:
Path: ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Aldgate East', 'Tower Hill', 'Aldgate', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Cost: 24
Expanded Nodes: 227

Baker Street to Wembley Park:
Path: ['Baker Street', 'Marylebone', 'Edgware Road', 'Paddington', 'Bayswater', 'Notting Hill Gate', 'Hollan

In [ ]:
def bfs(start_station, end_station, station_dict):
    """
    bfs

    Args:
    - start_station (str): The starting station.
    - end_station (str): The destination station.
    - station_dict (dict): The dictionary containing station connectivity.

    Returns:
    - Tuple: (path, total_cost, number_of_expanded_nodes)
    """
    visited = set()  # Keeps track of visited stations
    queue = deque([TubeState(start_station)])  # Queue of TubeState objects
    expanded_nodes = 0
    while queue:
        current_state = queue.popleft()
        current_station = current_state.current_station
        if current_station == end_station:
            return current_state.path, current_state.total_cost, expanded_nodes
        if current_station not in visited:
            visited.add(current_station)
            expanded_nodes += 1
            for new_state in current_state.expand(station_dict,line_dict):
                if new_state.current_station not in visited:
                    queue.append(new_state)

    return None, None, expanded_nodes

# Route from Euston to Victoria
bfs_path, bfs_total_cost, bfs_expanded_nodes = bfs("Euston", "Victoria", station_dict)
print("Euston to Victoria:")
print("Path:", bfs_path)
print("Total Cost:", bfs_total_cost)
print("Expanded Nodes Count:", bfs_expanded_nodes)
print()

# Route from Canada Water to Stratford
bfs_path, bfs_total_cost, bfs_expanded_nodes_count = bfs("Canada Water", "Stratford", station_dict)
print("Canada Water to Stratford:")
print("Path:", bfs_path)
print("Total Cost:", bfs_total_cost)
print("Expanded Nodes Count:", bfs_expanded_nodes)
print()

# Route from Baker Street to Wembley Park
bfs_path, bfs_total_cost, bfs_expanded_nodes_count = bfs("Baker Street", "Wembley Park", station_dict)
print("Baker Street to Wembley Park:")
print("Path:", bfs_path)
print("Total Cost:", bfs_total_cost)
print("Expanded Nodes Count:", bfs_expanded_nodes)
print()

# Route from New Cross Gate to Stepney Green
bfs_path, bfs_total_cost, bfs_expanded_nodes= bfs("New Cross Gate", "Stepney Green", station_dict)
print("New Cross Gate to Stepney Green:")
print("Path:", bfs_path)
print("Total Cost:", bfs_total_cost)
print("Expanded Nodes Count:", bfs_expanded_nodes)
print()

# Route from Ealing Broadway to South Kensington
bfs_path, bfs_total_cost, bfs_expanded_nodes_count = bfs("Ealing Broadway", "South Kensington", station_dict)
print("Ealing Broadway to South Kensington:")
print("Path:", bfs_path)
print("Total Cost:", bfs_total_cost)
print("Expanded Nodes Count:", bfs_expanded_nodes)

Euston to Victoria:
Path: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Total Cost: 7
Expanded Nodes Count: 34

Canada Water to Stratford:
Path: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Total Cost: 15
Expanded Nodes Count: 34

Baker Street to Wembley Park:
Path: ['Baker Street', 'Finchley Road', 'Wembley Park']
Total Cost: 13
Expanded Nodes Count: 34

New Cross Gate to Stepney Green:
Path: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Total Cost: 14
Expanded Nodes Count: 25

Ealing Broadway to South Kensington:
Path: ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Total Cost: 20
Expanded Nodes Count: 25


In [ ]:
def ucs(start_station, end_station, station_dict):
    """
    UCS

    Args:
    - start_station (str): The starting station.
    - end_station (str): The destination station.
    - station_dict (dict): The dictionary containing station connectivity and costs.

    Returns:
    - Tuple: (path, total_cost, number_of_expanded_nodes)
    """
    visited_nodes = {} # Keeps track of visited stations to avoid loop
    paths_to_explore = PriorityQueue()# Queue of TubeState objects
    paths_to_explore.put((0, [start_station]))
    expanded_nodes = 0

    while not paths_to_explore.empty():
        total_cost, path = paths_to_explore.get()
        current_station = path[-1]

        if current_station == end_station:
            return path, total_cost, expanded_nodes

        if current_station not in visited_nodes or visited_nodes[current_station] > total_cost:
            visited_nodes[current_station] = total_cost
            expanded_nodes += 1

            for neighbor, cost_to_neighbor in station_dict[current_station]:
                if neighbor not in visited_nodes:
                    new_cost = total_cost + cost_to_neighbor
                    new_path = path + [neighbor]
                    paths_to_explore.put((new_cost, new_path))

    return None, None, expanded_nodes

# Route from Euston to Victoria
ucs_path, ucs_cost, ucs_expanded_nodes = ucs("Euston", "Victoria", station_dict)
print("Euston to Victoria:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from Canada Water to Stratford
ucs_path, ucs_cost, ucs_expanded_nodes = ucs("Canada Water", "Stratford", station_dict)
print("Canada Water to Stratford:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from Baker Street to Wembley Park
ucs_path, ucs_cost, ucs_expanded_nodes = ucs("Baker Street", "Wembley Park", station_dict)
print("Baker Street to Wembley Park:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from New Cross Gate to Stepney Green
ucs_path, ucs_cost, ucs_expanded_nodes = ucs("New Cross Gate", "Stepney Green", station_dict)
print("New Cross Gate to Stepney Green:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from Ealing Broadway to South Kensington
ucs_path, ucs_cost, ucs_expanded_nodes = ucs("Ealing Broadway", "South Kensington", station_dict)
print("Ealing Broadway to South Kensington:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)

Euston to Victoria:
Path: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Cost: 7
Expanded Nodes: 34

Canada Water to Stratford:
Path: ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
Cost: 14
Expanded Nodes: 56

Baker Street to Wembley Park:
Path: ['Baker Street', 'Finchley Road', 'Wembley Park']
Cost: 13
Expanded Nodes: 81

New Cross Gate to Stepney Green:
Path: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Cost: 14
Expanded Nodes: 19

Ealing Broadway to South Kensington:
Path: ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Cost: 19
Expanded Nodes: 43


Extending the cost function


In [ ]:
def ucs_with_line_change(start_station, end_station, station_dict, line_dict, line_change_cost=2):
    """
    UCS with line change costs

    Args:
    - start_station (str): The starting station
    - end_station (str): The destination station
    - station_dict (dict): The dictionary containing station connectivity and costs
    - line_dict (dict): The dictionary containing the lines each station belongs to
    - line_change_cost (int): Additional cost for changing lines

    Returns:
    - Tuple: (path, total_cost, number_of_expanded_nodes)
    """
    visited_nodes = {}# Keeps track of visited stations to avoid loop
    paths_to_explore = PriorityQueue() # Queue of TubeState objects
    initial_state = TubeState(start_station, last_line=None)
    paths_to_explore.put((0, initial_state))
    expanded_nodes = 0

    while not paths_to_explore.empty():
        total_cost, current_state = paths_to_explore.get()
        current_station = current_state.current_station

        if current_station == end_station:
            return current_state.path, total_cost, expanded_nodes
        # Generate new states and add them to the queue
        if current_station not in visited_nodes or visited_nodes[current_station] > total_cost:
            visited_nodes[current_station] = total_cost
            expanded_nodes += 1
            #print(visited_nodes)

            for neighbor, cost_to_neighbor in station_dict[current_station]:
              if neighbor not in visited_nodes:
                  # Calculate the cost of changing lines, if applicable
                  new_last_line = current_state.determine_line_used(neighbor, line_dict)
                  line_change = line_change_cost if (new_last_line != current_state.last_line and current_state.last_line is not None) else 0
                  new_cost = total_cost + cost_to_neighbor + line_change
                  new_state = TubeState(neighbor, current_state.path + [neighbor], new_cost, current_state.visited_stations.union({neighbor}), new_last_line)

                  paths_to_explore.put((new_cost, new_state))

    return None, None, expanded_nodes

# Route from Euston to Victoria
ucs_path, ucs_cost, ucs_expanded_nodes = ucs_with_line_change("Euston", "Victoria", station_dict,line_dict)
print("Euston to Victoria:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from Canada Water to Stratford
ucs_path, ucs_cost, ucs_expanded_nodes = ucs_with_line_change("Canada Water", "Stratford", station_dict,line_dict)
print("Canada Water to Stratford:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from Baker Street to Wembley Park
ucs_path, ucs_cost, ucs_expanded_nodes = ucs_with_line_change("Baker Street", "Wembley Park", station_dict,line_dict)
print("Baker Street to Wembley Park:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from New Cross Gate to Stepney Green
ucs_path, ucs_cost, ucs_expanded_nodes = ucs_with_line_change("New Cross Gate", "Stepney Green", station_dict,line_dict)
print("New Cross Gate to Stepney Green:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)
print()

# Route from Ealing Broadway to South Kensington
ucs_path, ucs_cost, ucs_expanded_nodes = ucs_with_line_change("Ealing Broadway", "South Kensington", station_dict,line_dict)
print("Ealing Broadway to South Kensington:")
print("Path:", ucs_path)
print("Cost:", ucs_cost)
print("Expanded Nodes:", ucs_expanded_nodes)

Euston to Victoria:
Path: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Cost: 7
Expanded Nodes: 22

Canada Water to Stratford:
Path: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Cost: 15
Expanded Nodes: 43

Baker Street to Wembley Park:
Path: ['Baker Street', 'Finchley Road', 'Wembley Park']
Cost: 13
Expanded Nodes: 63

New Cross Gate to Stepney Green:
Path: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Cost: 16
Expanded Nodes: 15

Ealing Broadway to South Kensington:
Path: ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Cost: 19
Expanded Nodes: 41


Heuristic search

In [ ]:
def zone_heuristic(current_station, target_station, zone_dict):
    current_zones = {int(zone) for zone in zone_dict[current_station]}
    target_zones = {int(zone) for zone in zone_dict[target_station]}
    return min(abs(cz - tz) for cz in current_zones for tz in target_zones)

def heuristic_bfs(start_station, end_station, station_dict, zone_dict, line_dict, line_change_cost=2):
    """
    Heuristic BFS

    Args:
    - start_station (str): The starting station.
    - end_station (str): The destination station.
    - station_dict (dict): The dictionary containing station connectivity and costs.
    - zone_dict (dict): The dictionary containing the zones each station belongs to.
    - line_dict (dict): The dictionary containing the lines each station belongs to.
    - line_change_cost (int): Additional cost for changing lines.

    Returns:
    - Tuple: (path, total_cost, number_of_expanded_nodes)
    """
    visited_nodes = {}
    paths_to_explore = PriorityQueue()
    initial_state = TubeState(start_station, last_line=None)
    paths_to_explore.put((0, initial_state))
    expanded_nodes = 0

    while not paths_to_explore.empty():
        _, current_state = paths_to_explore.get()
        current_station = current_state.current_station

        if current_station == end_station:
            return current_state.path, current_state.total_cost, expanded_nodes

        if current_station not in visited_nodes:
            visited_nodes[current_station] = current_state.total_cost
            expanded_nodes += 1

            for neighbor, cost_to_neighbor in station_dict[current_station]:
                if neighbor not in visited_nodes:
                    new_last_line = current_state.determine_line_used(neighbor, line_dict)
                    line_change = line_change_cost if (new_last_line != current_state.last_line and current_state.last_line is not None) else 0
                    new_cost = current_state.total_cost + cost_to_neighbor + line_change

                    # Heuristic value is based on zone proximity
                    heuristic_value = zone_heuristic(neighbor, end_station, zone_dict)

                    # The priority in the queue is based on heuristic value
                    new_state = TubeState(neighbor, current_state.path + [neighbor], new_cost, current_state.visited_stations.union({neighbor}), new_last_line)
                    paths_to_explore.put((heuristic_value, new_state))

    return None, None, expanded_nodes

# New Cross Gate to Stepney Green:
bfs_path, bfs_cost, bfs_expanded_nodes = heuristic_bfs("New Cross Gate", "Stepney Green", station_dict, zone_dict, line_dict)
print("New Cross Gate to Stepney Green:")
print("Path:", bfs_path)
print("Total Cost:", bfs_cost)
print("Number of Expanded Nodes:", bfs_expanded_nodes)
print()
#Ealing Broadway to South Kensington:
bfs_path, bfs_cost, bfs_expanded_nodes = heuristic_bfs("Ealing Broadway", "South Kensington", station_dict, zone_dict, line_dict)
print("Ealing Broadway to South Kensington:")
print("Path:", bfs_path)
print("Total Cost:", bfs_cost)
print("Number of Expanded Nodes:", bfs_expanded_nodes)
print()

New Cross Gate to Stepney Green:
Path: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Total Cost: 16
Number of Expanded Nodes: 12

Ealing Broadway to South Kensington:
Path: ['Ealing Broadway', 'West Acton', 'North Acton', 'East Acton', 'White City', "Shepherd's Bush", 'Holland Park', 'Notting Hill Gate', 'High Street Kensington', 'Gloucester Road', 'South Kensington']
Total Cost: 27
Number of Expanded Nodes: 18



### Genetic Algorithm


In [ ]:
import random
import string
import hashlib
import math
import numpy as np

# Provided Functions
def get_password(student_username, l=10):
    options = string.digits + string.ascii_uppercase + "_"
    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
        s += options[n % len(options)]
    return s[0:l]

def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score

MAX_VALUE = distance_function('0000000000', '__________')

def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    for phrase in list_of_phrases:
        ordered_dict[phrase] = 1 - distance_function(phrase, student_password) / MAX_VALUE
    return ordered_dict

# GA Implementation
# Initialize a population of individuals with random genes
def initialize_population(size, length):
    population = []
    chars = string.ascii_uppercase + string.digits + "_"
    for _ in range(size):
        individual = ''.join(random.choice(chars) for _ in range(length))
        population.append(individual)
    return population
# recombination between two parents to create two children
def select_parents(population, fitnesses, num_parents):
    parents = []
    for _ in range(num_parents):
        candidates = random.sample(population, k=2)
        parent = max(candidates, key=lambda x: fitnesses[x])
        parents.append(parent)
    return parents
#a specified number of parents based on their fitness values
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2
# Apply mutations to an individual with a given mutation
def mutate(individual, mutation_rate):
    chars = string.ascii_uppercase + string.digits + "_"
    individual = list(individual)
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = random.choice(chars)
    return ''.join(individual)

def genetic_algorithm(student_username, population_size, generations, mutation_rate):
    student_password = get_password(student_username)
    population = initialize_population(population_size, len(student_password))
    # Calculate fitness for each candidate and sellect parents based on fitness
    for generation in range(generations):
        fitnesses = get_normalised_fitness(population, student_password)
        parents = select_parents(population, fitnesses, population_size // 2)
        next_population = parents.copy()
        #generate new individuals by combining genes from parent individuals through crossover
        while len(next_population) < population_size:
            parent1, parent2 = random.choice(parents), random.choice(parents)
            child1, child2 = crossover(parent1, parent2)
            next_population.append(mutate(child1, mutation_rate))
            next_population.append(mutate(child2, mutation_rate))

        population = next_population
        fitnesses = get_normalised_fitness(population, student_password)
        # Check for an exact match and return if found
        for individual in population:
            if fitnesses[individual] == 1.0:
                return individual,generation
    # Return the best found if exact match not found
    return max(population, key=lambda x: fitnesses[x])


# Run the GA
student_username = "ec23425"
student_password = get_password(student_username)
found_password,_ = genetic_algorithm(student_username, 100, 1000, 0.01)
print("Actual Password:", student_password)
print("Found Password by GA:", found_password)

Actual Password: LRREVSQ_6S
Found Password by GA: LRREVSQ_6S


Number of reproductions


In [ ]:
# Parameters
student_username = "ec23425"
population_size = 100
max_generations = 1000
mutation_rate = 0.01
num_runs = 10

# Run the GA multiple times and record the number of generations
generations_needed = []
for _ in range(num_runs):
    idividual,generations = genetic_algorithm(student_username, population_size, max_generations, mutation_rate)
    generations_needed.append(generations)

# Calculate the average and standard deviation
average_generations = np.mean(generations_needed)
std_dev_generations = np.std(generations_needed)

print("Average number of generations:", average_generations)
print("Standard deviation:", std_dev_generations)


Average number of generations: 378.0
Standard deviation: 131.14114533585558


Hyperparameters

In [ ]:
# Function to run the genetic algorithm
def hyperparameter(student_username, mutation_rate, num_runs, max_generations, population_size):
    generations_needed = []
    for _ in range(num_runs):
        idividual , generations = genetic_algorithm(student_username, population_size, max_generations, mutation_rate)
        generations_needed.append(generations)
    return np.mean(generations_needed), np.std(generations_needed)

# Parameters
student_username = "ec23425"
population_size = 100
max_generations = 1000
num_runs = 10
mutation_rates = [0.01, 0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2]

#Run the genetic algorithm for each mutation rate and collect results
results = {}
for rate in mutation_rates:
    avg, std = hyperparameter(student_username, rate, num_runs, max_generations, population_size)
    results[rate] = {"Average Generations": avg, "Standard Deviation": std}

# Print the results in a formatted table
header = ["Mutation Rate", "Average Generations", "Standard Deviation"]
print("{:<15} {:<20} {:<20}".format(*header))
for rate, metrics in results.items():
    print("{:<15} {:<20} {:<20}".format(rate, round(metrics['Average Generations'], 2), round(metrics['Standard Deviation'], 2)))


Mutation Rate   Average Generations  Standard Deviation  
0.01            372.0                94.22               
0.02            241.6                104.56              
0.03            144.2                31.81               
0.04            123.6                40.2                
0.05            127.1                34.81               
0.06            139.5                47.09               
0.07            110.2                30.33               
0.08            143.6                42.82               
0.09            151.5                75.72               
0.1             115.6                45.06               
0.2             345.5                111.69              
